In [1]:
import pandas as pd
import numpy as np
from data_work import processed
df=pd.read_csv("/Users/mac/Dev/data/dta_IoT/new2test.csv")
#delete all null columns
id=[16,17,21,22,23,24]
col=df.columns
for idx in id:
    df=df.drop(col[idx],axis=1)
data=processed(df,"subcategory ")

In [2]:
data

,subcategory,pkSeqID,stime,flgs,proto,saddr,sport,daddr,dport,pkts,...,max,spkts,dpkts,sbytes,dbytes,rate,srate,drate,attack,category
0,2,3711,6284,0,2,5,4415,17,600,1,...,130,0,1,2,1,2348,0,0,1,3
1,3,1516,1428,0,3,5,3130,44,697,1,...,1151,0,1,7,86,1534,0,0,0,1
2,7,6150,3656,0,3,0,432,25,901,9,...,2171,9,0,71,0,368,552,0,1,0
3,4,2083,1903,0,2,3,3079,20,847,1,...,191,0,1,0,1,2279,0,0,1,2
4,3,1677,1588,0,3,3,3278,27,697,1,...,0,1,0,41,0,525,710,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7021,6,5814,3323,7,2,2,2679,25,901,6,...,0,6,0,424,0,104,34,0,1,0
7022,5,713,694,0,2,3,1908,20,948,1,...,46,0,1,0,1,2456,0,0,1,2
7023,0,3961,5427,0,2,3,1966,20,372,27,...,2391,14,15,462,130,1611,1288,1062,1,3
7024,3,1872,1782,0,3,11,1883,20,731,65,...,2531,51,0,538,0,1694,1428,0,0,1


In [4]:
import torch
data=torch.tensor(data.values)

In [ ]:
label_counts = torch.sum(labels_one_hot, dim=0)
num_labels = label_counts.size(0)
for label_idx in range(num_labels):
    label_count = label_counts[label_idx].item()
    print(f"Label {label_idx}: {label_count} samples")

In [ ]:
data=torch.tensor(data.values)

In [ ]:
data


In [ ]:
x

In [ ]:
embed=nn.Embedding(8,8)
y=data[:32,:1]
x=data[:32,1:]
print(x.size())
print(y.size())
tes=torch.cat((embed(y).squeeze(),x),1)
tes.size()

In [ ]:
data.size()

In [20]:
import numpy as np

def onehot_encode(labels, num_classes):
    num_samples = len(labels)
    encoded_labels = np.zeros((num_samples, num_classes))
    for i in range(num_samples):
        label = labels[i]
        encoded_labels[i, label] = 1
    return encoded_labels

In [40]:
import utils, torch, time, os, pickle
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.autograd import Variable

class generator(nn.Module):
    def __init__(self, args):
        super(generator, self).__init__()
        self.input_dim = args.noise_size
        self.output_dim = args.n_features
        self.class_num = args.n_classes
        self.label_emb = nn.Embedding(self.class_num,self.class_num)

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers
    
        self.model = nn.Sequential(
            *block(self.input_dim + self.class_num, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024,self.output_dim)
        )

    def forward(self, noise ,label):
        x = torch.cat((self.label_emb(label).squeeze(),noise), 1)
        x = self.model(x)
        return x

class discriminator(nn.Module):
    def __init__(self,args):
        super(discriminator, self).__init__()
        self.input_dim = args.n_features
        self.output_dim = args.n_features
        self.class_num = args.n_classes
        self.label_emb = nn.Embedding(self.class_num,self.class_num)

        self.model = nn.Sequential(
            nn.Linear((self.class_num + self.input_dim), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256)
        )
        self.adv_layer = nn.Sequential(nn.Linear(256, 1), nn.Sigmoid())
        self.aux_layer = nn.Sequential(nn.Linear(256, self.class_num), nn.Softmax())
        

    def forward(self, input ,label):
        # Concatenate label embedding and image to produce input
        x = torch.cat((self.label_emb(label).squeeze(), input), 1)
        x=self.model(x)
        real=self.adv_layer(x)
        label=self.aux_layer(x)
        return real,label 
    
class CGAN(object):
    def __init__(self, args):
        # parameters
        self.epoch = args.epoch
        self.batch_size = args.batch_size
        self.save_dir = args.save_dir
        self.result_dir = args.result_dir
        self.dataset = args.dataset
        self.log_dir = args.log_dir
        self.gpu_mode = args.gpu_mode
        self.model_name = args.gan_type
        self.z_dim = args.z_dim
        self.class_num = args.n_class
        # load dataset
        self.data_loader = DataLoader(self.dataset, batch_size=self.batch_size, shuffle=True)
        data=next(iter(self.data_loader))
        #option
        dim=int(data.shape[1])-8

        # networks init
        self.G = generator(args)
        self.D = discriminator(args)
        self.G_optimizer = optim.RMSprop(self.G.parameters(), lr=args.lrG, alpha=0.9)
        self.D_optimizer = optim.RMSprop(self.D.parameters(), lr=args.lrD, alpha=0.9)
        
        self.MSE_loss = torch.nn.MSELoss()
        self.BCE_loss=nn.BCELoss()
        # Loss functions
        self.adversarial_loss = torch.nn.BCELoss()
        self.auxiliary_loss = torch.nn.CrossEntropyLoss()
        print('---------- Networks architecture -------------')
        utils.print_network(self.G)
        utils.print_network(self.D)
        print('-----------------------------------------------')



    def train(self):
        self.train_hist = {}
        self.train_hist['D_loss'] = []
        self.train_hist['G_loss'] = []
        self.train_hist['per_epoch_time'] = []
        self.train_hist['total_time'] = []

        self.y_real_ = Variable(torch.FloatTensor(self.batch_size, 1).fill_(1.0), requires_grad=False)
        self.y_fake_ = Variable(torch.FloatTensor(self.batch_size, 1).fill_(0.0), requires_grad=False)

        #self.y_real_, self.y_fake_ = torch.ones(self.batch_size, 1), torch.zeros(self.batch_size, 1)



        self.D.train()
        print('training start!!')
        start_time = time.time()
        for epoch in range(self.epoch):
            self.G.train()
            epoch_start_time = time.time()
            for iter, da in enumerate(self.data_loader):
                x_=da[:,1:].float()
                y_=da[:,:1].int()
                if iter == self.data_loader.dataset.__len__() // self.batch_size:
                    break

                z_ = Variable(torch.FloatTensor(np.random.normal(0, 1, (self.batch_size, self.z_dim))))
                #y_check= onehot_encode(y_,8)
                #y_check=torch.tensor(y_check)
                y_check=y_
                # update D network
                self.D.train()
                self.G.eval()
                self.D_optimizer.zero_grad()


                real,label = self.D(x_, y_)
                #D_real_loss = self.MSE_loss(D_real, self.y_real_)
                G_ = self.G(z_, y_)
                fake,label_ = self.D(G_, y_)
                #D_fake_loss = self.MSE_loss(D_fake, self.y_fake_)
                D_fake_loss=(self.adversarial_loss(fake, self.y_fake_) + self.auxiliary_loss(label_, y_check)) / 2
                D_real_loss=(self.adversarial_loss(real, self.y_real_) + self.auxiliary_loss(label, y_check)) / 2


                D_loss = D_real_loss + D_fake_loss
                self.train_hist['D_loss'].append(D_loss.item())

                D_loss.backward()
                self.D_optimizer.step()

                 # update G network
                self.D.eval()
                self.G.train()
                self.G_optimizer.zero_grad()

                z_=torch.rand((self.batch_size, self.z_dim))
                G_ = self.G(z_, y_)
                real,label = self.D(G_, y_)
                #G_loss = self.MSE_loss(D_fake, self.y_real_)

                G_loss=0.5 * (self.adversarial_loss(real, self.y_real_) + self.auxiliary_loss(label, y_check))
                self.train_hist['G_loss'].append(G_loss.item())

                G_loss.backward()
                self.G_optimizer.step()

               
                if (iter + 1) == self.data_loader.dataset.__len__() // self.batch_size:
                    print("Epoch: [%2d] [%4d/%4d] D_loss: %.8f, G_loss: %.8f" %
                          ((epoch + 1), (iter + 1), self.data_loader.dataset.__len__() // self.batch_size, D_loss.item(), G_loss.item()))

            self.train_hist['per_epoch_time'].append(time.time() - epoch_start_time)
                

        self.train_hist['total_time'].append(time.time() - start_time)
        print("Avg one epoch time: %.2f, total %d epochs time: %.2f" % (np.mean(self.train_hist['per_epoch_time']),
              self.epoch, self.train_hist['total_time'][0]))
        print("Training finish!")


In [41]:
class Args:
    def __init__(self,data):
        self.epoch = 10
        self.batch_size = 32
        self.save_dir = '/Users/mac/Dev/source/article/FerderatedLearning/federated_learning/NonIID/model.h5'
        self.result_dir = '/Users/mac/Dev/source/article/FerderatedLearning/federated_learning/NonIID/result.txt'
        self.dataset = data
        self.log_dir = 'logs/'
        self.gpu_mode = True
        self.gan_type = 'cGAN'
        self.z_dim = 10
        self.n_class = 8
        self.sample_num = self.n_class ** 2
        self.lrG=0.0001
        self.lrD=0.0001
        self.n_epochs=self.n_class
        self.n_classes=self.n_class
        self.noise_size=self.z_dim
        self.n_features=28

In [42]:
arg=Args(data)
print(arg.dataset)

tensor([[   2, 3711, 6284,  ...,    0,    1,    3],
        [   3, 1516, 1428,  ...,    0,    0,    1],
        [   7, 6150, 3656,  ...,    0,    1,    0],
        ...,
        [   0, 3961, 5427,  ..., 1062,    1,    3],
        [   3, 1872, 1782,  ...,    0,    0,    1],
        [   2, 3827, 6400,  ..., 1247,    1,    3]])


In [43]:
model=CGAN(arg)

---------- Networks architecture -------------
generator(
  (label_emb): Embedding(8, 8)
  (model): Sequential(
    (0): Linear(in_features=18, out_features=128, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Linear(in_features=128, out_features=256, bias=True)
    (3): BatchNorm1d(256, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Linear(in_features=256, out_features=512, bias=True)
    (6): BatchNorm1d(512, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Linear(in_features=512, out_features=1024, bias=True)
    (9): BatchNorm1d(1024, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Linear(in_features=1024, out_features=28, bias=True)
  )
)
Total number of parameters: 724700
discriminator(
  (label_emb): Embedding(8, 8)
  (model): S

In [44]:
model.train()

training start!!


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/nn/modules/container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


RuntimeError: 0D or 1D target tensor expected, multi-target not supported

In [31]:
y=torch.tensor([[1],[2],[3],[4],[5],[6],[7],[0],[1],[2],[3],[4],[5],[6],[7],[0],[1],[2],[3],[4],[5],[6],[7],[0]])




In [32]:
y

tensor([[1],
        [2],
        [3],
        [4],
        [5],
        [6],
        [7],
        [0],
        [1],
        [2],
        [3],
        [4],
        [5],
        [6],
        [7],
        [0],
        [1],
        [2],
        [3],
        [4],
        [5],
        [6],
        [7],
        [0]])

In [33]:
len=y.shape[0]
len

24

In [34]:
z= torch.rand(len,10)
y=torch.tensor(y)

/var/folders/0j/2h4f0bj12r775fg_s2v4hpqh0000gn/T/ipykernel_89047/1650534317.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y=torch.tensor(y)


In [35]:
mm=torch.cat([z,y],1)

In [36]:
mm

tensor([[6.2243e-01, 1.0103e-01, 2.1562e-01, 6.5246e-01, 2.4079e-01, 8.1744e-01,
         3.4712e-01, 3.8573e-01, 7.0011e-01, 6.8769e-01, 1.0000e+00],
        [1.3400e-01, 6.7474e-01, 6.0900e-01, 6.1009e-01, 8.1475e-01, 9.4652e-01,
         9.5120e-01, 9.6646e-01, 6.2355e-01, 8.9124e-01, 2.0000e+00],
        [5.2764e-01, 9.7205e-01, 8.7093e-01, 5.4650e-01, 3.7128e-01, 5.0396e-01,
         9.7573e-01, 7.4810e-01, 1.0209e-01, 4.4495e-02, 3.0000e+00],
        [1.5447e-01, 5.1976e-01, 4.6833e-01, 7.2069e-01, 2.5893e-01, 2.8095e-01,
         5.8347e-01, 5.3661e-01, 9.7970e-01, 1.8402e-01, 4.0000e+00],
        [2.9829e-02, 6.3900e-01, 1.1078e-01, 7.5116e-03, 6.1060e-02, 7.6196e-01,
         1.6047e-01, 6.0523e-01, 9.1689e-01, 7.0519e-01, 5.0000e+00],
        [6.2035e-01, 5.1903e-01, 7.1086e-01, 5.8471e-01, 3.1769e-01, 3.1155e-01,
         8.8932e-01, 5.8373e-03, 2.0105e-01, 7.6207e-01, 6.0000e+00],
        [1.8573e-01, 9.0780e-01, 8.0976e-01, 4.6425e-01, 2.7944e-01, 2.1434e-01,
         1.79

In [37]:
mm.shape[1]

11

In [38]:
data_fake=model.G(z,y)

In [39]:
data_fake

tensor([[ 9.7018e+01,  1.0032e+02,  1.7561e+00,  4.3638e-02,  2.5480e+00,
          3.1089e+01,  1.8573e+00,  1.0897e+01,  3.0790e+00,  3.4829e+00,
         -1.7003e+00,  9.4960e+01,  1.5222e+01,  3.4034e+01,  2.5746e+01,
          3.4629e-01,  1.4285e+01,  8.5204e+00,  2.1589e+01, -5.2481e+00,
          3.0474e+00,  2.1635e+00,  4.4235e+00,  2.6075e+01,  1.4781e+01,
          4.8550e+00, -9.5278e-01,  5.2621e+00],
        [ 6.2846e+01,  4.7878e+01,  8.5074e-01,  1.8941e-02, -2.4761e+00,
          1.0175e+02,  3.2814e+00,  3.1476e+01, -1.3504e+00,  7.9025e+00,
          2.3773e+00,  5.7999e+01,  3.4034e+01,  1.3652e+01,  1.8854e-01,
         -5.9643e+00,  5.9182e+00,  1.1616e+01,  6.3129e+00, -1.3573e-01,
          1.1876e+00,  8.9249e-01, -1.0012e+00,  8.7436e+01, -8.2863e-01,
          2.1164e+00, -2.9009e+00, -3.3203e+00],
        [ 2.4500e+01,  1.3391e+01, -1.5957e-01,  1.3286e+00, -4.6580e-01,
          4.4691e+01,  9.9341e-01,  1.7558e+01,  1.2243e+00, -1.0922e+00,
          3.15